# Spark SQL API

In [3]:
from pyspark import SparkConf
from pyspark import SparkContext

In [4]:
import os
PATH_TO_FILES = '/home/dmitry/pyspark-training/files'
MASTER_URL = 'spark://185739.simplecloud.ru:7077'

In [5]:
conf = SparkConf()
conf = conf.set('spark.driver.memory', '512m')
conf = conf.set('spark.executor.memory', '512m')
conf = conf.set('spark.executor.cores', '2')

In [6]:
sc = SparkContext(MASTER_URL, appName="SQL API", conf=conf)

In [7]:
from pyspark.sql import SparkSession

In [8]:
sql = SparkSession(sc)

In [9]:
sql

In [10]:
data = sql.read.json(os.path.join(PATH_TO_FILES, 'character.json'))

In [11]:
data.printSchema()

root
 |-- created: string (nullable = true)
 |-- episode: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- gender: string (nullable = true)
 |-- id: long (nullable = true)
 |-- image: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- origin: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- species: string (nullable = true)
 |-- status: string (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)



In [12]:
data

DataFrame[created: string, episode: array<string>, gender: string, id: bigint, image: string, location: struct<name:string,url:string>, name: string, origin: struct<name:string,url:string>, species: string, status: string, type: string, url: string]

In [13]:
data.columns

['created',
 'episode',
 'gender',
 'id',
 'image',
 'location',
 'name',
 'origin',
 'species',
 'status',
 'type',
 'url']

In [14]:
data.registerTempTable('rick_and_morty_characters')

In [15]:
sql.sql("select name, gender from rick_and_morty_characters").show()

+--------------------+-------+
|                name| gender|
+--------------------+-------+
|        Rick Sanchez|   Male|
|         Morty Smith|   Male|
|        Summer Smith| Female|
|          Beth Smith| Female|
|         Jerry Smith|   Male|
|Abadango Cluster ...| Female|
|    Abradolf Lincler|   Male|
|    Adjudicator Rick|   Male|
|     Agency Director|   Male|
|          Alan Rails|   Male|
|     Albert Einstein|   Male|
|           Alexander|   Male|
|        Alien Googah|unknown|
|         Alien Morty|   Male|
|          Alien Rick|   Male|
|        Amish Cyborg|   Male|
|               Annie| Female|
|       Antenna Morty|   Male|
|        Antenna Rick|   Male|
|Ants in my Eyes J...|   Male|
+--------------------+-------+



In [19]:
sql.sql("""
    SELECT 
        name, 
        gender,
        status
    FROM 
        rick_and_morty_characters
    WHERE
        gender='Male'
    ORDER BY name DESC
""").show(100)

+--------------------+------+-------+
|                name|gender| status|
+--------------------+------+-------+
|        Rick Sanchez|  Male|  Alive|
|         Morty Smith|  Male|  Alive|
|         Jerry Smith|  Male|  Alive|
|Ants in my Eyes J...|  Male|unknown|
|        Antenna Rick|  Male|unknown|
|       Antenna Morty|  Male|  Alive|
|        Amish Cyborg|  Male|   Dead|
|          Alien Rick|  Male|unknown|
|         Alien Morty|  Male|unknown|
|           Alexander|  Male|   Dead|
|     Albert Einstein|  Male|   Dead|
|          Alan Rails|  Male|   Dead|
|     Agency Director|  Male|   Dead|
|    Adjudicator Rick|  Male|   Dead|
|    Abradolf Lincler|  Male|unknown|
+--------------------+------+-------+



In [21]:
sql.sql("""
    SELECT
        *
    FROM
        (SELECT 
            name, 
            gender,
            status
        FROM 
            rick_and_morty_characters
        WHERE
            gender='Male'
        ORDER BY name DESC) as t1
    INNER JOIN
        (SELECT 
            name, 
            gender,
            status
        FROM 
            rick_and_morty_characters
        WHERE
            gender='Male'
        ORDER BY name DESC) as t2
    ON t1.name = t2.name
        
""").show(100)

+--------------------+------+-------+--------------------+------+-------+
|                name|gender| status|                name|gender| status|
+--------------------+------+-------+--------------------+------+-------+
|        Rick Sanchez|  Male|  Alive|        Rick Sanchez|  Male|  Alive|
|         Morty Smith|  Male|  Alive|         Morty Smith|  Male|  Alive|
|         Jerry Smith|  Male|  Alive|         Jerry Smith|  Male|  Alive|
|Ants in my Eyes J...|  Male|unknown|Ants in my Eyes J...|  Male|unknown|
|        Antenna Rick|  Male|unknown|        Antenna Rick|  Male|unknown|
|       Antenna Morty|  Male|  Alive|       Antenna Morty|  Male|  Alive|
|        Amish Cyborg|  Male|   Dead|        Amish Cyborg|  Male|   Dead|
|          Alien Rick|  Male|unknown|          Alien Rick|  Male|unknown|
|         Alien Morty|  Male|unknown|         Alien Morty|  Male|unknown|
|           Alexander|  Male|   Dead|           Alexander|  Male|   Dead|
|     Albert Einstein|  Male|   Dead| 

In [22]:
from pyspark.sql import functions as F

In [27]:
data.where(F.col('gender') == 'Male').orderBy(F.col('name').desc()).select('name', 'gender', 'status').show()

+--------------------+------+-------+
|                name|gender| status|
+--------------------+------+-------+
|        Rick Sanchez|  Male|  Alive|
|         Morty Smith|  Male|  Alive|
|         Jerry Smith|  Male|  Alive|
|Ants in my Eyes J...|  Male|unknown|
|        Antenna Rick|  Male|unknown|
|       Antenna Morty|  Male|  Alive|
|        Amish Cyborg|  Male|   Dead|
|          Alien Rick|  Male|unknown|
|         Alien Morty|  Male|unknown|
|           Alexander|  Male|   Dead|
|     Albert Einstein|  Male|   Dead|
|          Alan Rails|  Male|   Dead|
|     Agency Director|  Male|   Dead|
|    Adjudicator Rick|  Male|   Dead|
|    Abradolf Lincler|  Male|unknown|
+--------------------+------+-------+



In [33]:
part = data.where(F.col('gender') == 'Male').orderBy(F.col('name').desc()).select('name', 'gender', 'status')
part

DataFrame[name: string, gender: string, status: string]

In [34]:
part.persist()

DataFrame[name: string, gender: string, status: string]

In [36]:
part.join(part, on=['name', ], how='inner').show()

+--------------------+------+-------+------+-------+
|                name|gender| status|gender| status|
+--------------------+------+-------+------+-------+
|        Rick Sanchez|  Male|  Alive|  Male|  Alive|
|         Morty Smith|  Male|  Alive|  Male|  Alive|
|         Jerry Smith|  Male|  Alive|  Male|  Alive|
|Ants in my Eyes J...|  Male|unknown|  Male|unknown|
|        Antenna Rick|  Male|unknown|  Male|unknown|
|       Antenna Morty|  Male|  Alive|  Male|  Alive|
|        Amish Cyborg|  Male|   Dead|  Male|   Dead|
|          Alien Rick|  Male|unknown|  Male|unknown|
|         Alien Morty|  Male|unknown|  Male|unknown|
|           Alexander|  Male|   Dead|  Male|   Dead|
|     Albert Einstein|  Male|   Dead|  Male|   Dead|
|          Alan Rails|  Male|   Dead|  Male|   Dead|
|     Agency Director|  Male|   Dead|  Male|   Dead|
|    Adjudicator Rick|  Male|   Dead|  Male|   Dead|
|    Abradolf Lincler|  Male|unknown|  Male|unknown|
+--------------------+------+-------+------+--

In [32]:
part.join(part, on=part['name'] == part['name'], how='inner').show() # копируются колонки из обоих операндов

+--------------------+------+-------+--------------------+------+-------+
|                name|gender| status|                name|gender| status|
+--------------------+------+-------+--------------------+------+-------+
|        Rick Sanchez|  Male|  Alive|        Rick Sanchez|  Male|  Alive|
|         Morty Smith|  Male|  Alive|         Morty Smith|  Male|  Alive|
|         Jerry Smith|  Male|  Alive|         Jerry Smith|  Male|  Alive|
|Ants in my Eyes J...|  Male|unknown|Ants in my Eyes J...|  Male|unknown|
|        Antenna Rick|  Male|unknown|        Antenna Rick|  Male|unknown|
|       Antenna Morty|  Male|  Alive|       Antenna Morty|  Male|  Alive|
|        Amish Cyborg|  Male|   Dead|        Amish Cyborg|  Male|   Dead|
|          Alien Rick|  Male|unknown|          Alien Rick|  Male|unknown|
|         Alien Morty|  Male|unknown|         Alien Morty|  Male|unknown|
|           Alexander|  Male|   Dead|           Alexander|  Male|   Dead|
|     Albert Einstein|  Male|   Dead| 

In [37]:
part.unpersist()

DataFrame[name: string, gender: string, status: string]

In [38]:
new_data = part.join(part, on=['name', ], how='inner')

In [40]:
new_data.explain(True) # логический план

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(name))
:- Project [name#12, gender#8, status#15]
:  +- Sort [name#12 DESC NULLS LAST], true
:     +- Filter (gender#8 = Male)
:        +- Relation[created#6,episode#7,gender#8,id#9L,image#10,location#11,name#12,origin#13,species#14,status#15,type#16,url#17] json
+- Project [name#556, gender#552, status#559]
   +- Sort [name#556 DESC NULLS LAST], true
      +- Filter (gender#552 = Male)
         +- Relation[created#550,episode#551,gender#552,id#553L,image#554,location#555,name#556,origin#557,species#558,status#559,type#560,url#561] json

== Analyzed Logical Plan ==
name: string, gender: string, status: string, gender: string, status: string
Project [name#12, gender#8, status#15, gender#552, status#559]
+- Join Inner, (name#12 = name#556)
   :- Project [name#12, gender#8, status#15]
   :  +- Sort [name#12 DESC NULLS LAST], true
   :     +- Filter (gender#8 = Male)
   :        +- Relation[created#6,episode#7,gender#8,id#9L,image#10,lo

In [39]:
new_data.explain(False) # только физический план

== Physical Plan ==
*(4) Project [name#12, gender#8, status#15, gender#552, status#559]
+- *(4) BroadcastHashJoin [name#12], [name#556], Inner, BuildRight
   :- *(4) Project [name#12, gender#8, status#15]
   :  +- *(4) Sort [name#12 DESC NULLS LAST], true, 0
   :     +- Exchange rangepartitioning(name#12 DESC NULLS LAST, 200)
   :        +- *(1) Project [gender#8, name#12, status#15]
   :           +- *(1) Filter ((isnotnull(gender#8) && (gender#8 = Male)) && isnotnull(name#12))
   :              +- *(1) FileScan json [gender#8,name#12,status#15] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/home/dmitry/pyspark-training/files/character.json], PartitionFilters: [], PushedFilters: [IsNotNull(gender), EqualTo(gender,Male), IsNotNull(name)], ReadSchema: struct<gender:string,name:string,status:string>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true]))
      +- *(3) Project [name#556, gender#552, status#559]
         +- *(3) Sort [name#556 DES

In [41]:
sql.stop()

In [42]:
sc.stop()